In [1]:
from pyntcloud import PyntCloud
from pyntcloud.geometry.areas import *

import numpy as np
import pandas as pd

from rotations import *
from sklearn import preprocessing

In [2]:

#diamond = PyntCloud.from_file('cube.off')
diamond = PyntCloud.from_file('bed_0001.off')

v1, v2, v3 = diamond.get_mesh_vertices(False, False)

v1_xyz = v1[:, :3]
v2_xyz = v2[:, :3]
v3_xyz = v3[:, :3]

areas = triangle_area_multi(v1_xyz, v2_xyz, v3_xyz)
probabilities = areas / np.sum(areas)

v1 = v1_xyz[4] #Choosing an arbitrary face to test on
v2 = v2_xyz[4]
v3 = v3_xyz[4]

#choosing a starting point on our face
u = np.random.rand(1, 1)
v = (np.random.rand(1, 1)) * (1-u)

result = pd.DataFrame()

result_xyz = (v1 * u) + (v2 * v) + ((1 - (u + v)) * v3)
starting_point = result_xyz.astype(np.float32)

result["x"] = starting_point[:, 0]
result["y"] = starting_point[:, 1]
result["z"] = starting_point[:, 2]

In [3]:
print(starting_point)
print(v1)

[[ -2.145118  39.936115 -21.      ]]
[ 31.75      40.972496 -21.      ]


In [4]:
# Translate face such that starting_point is the origin
translation = starting_point
v1,v2,v3,starting_point = [np.subtract(p, translation) for p in (v1,v2,v3,starting_point)] # make list or dict eventually

print(v1[0],v2,v3,starting_point[0])

[33.89512    1.0363808  0.       ] [[-29.604881    -0.96361923   0.        ]] [[-29.604881    1.0363808   0.       ]] [0. 0. 0.]


In [5]:
#find angle to rotate by around the normal
theta = np.random.uniform(0, 2*np.pi)
theta = (theta * 180)/np.pi
#find the point to rotate around the normal
dist = np.linalg.norm(v1-(starting_point[0]))
#print(dist)

r = 0.4
rho = np.random.uniform(np.cbrt(r), np.cbrt(2*r))
#print(rho)
parts = (dist/rho) 
#print(np.ceil(parts))

points = list(zip(np.linspace((starting_point[0])[0], v1[0][0], parts+1), np.linspace((starting_point[0])[1],v1[0][1], parts+1), np.linspace((starting_point[0])[2], v1[0][2], parts+1)))

pre_rotation_point = points[1]   # finds a point rho distance from the starting point to then apply rotation



AB = np.subtract(v2[0],v1[0])
AC = np.subtract(v3[0],v1[0])


normal = np.cross(AB, AC)
#normal = preprocessing.normalize([normal], norm='l2')   #

print(pre_rotation_point)
new = vrotate(pre_rotation_point, theta, normal)
print(new)

(0.7532248602973091, 0.02303068372938368, 0.0)
[ 0.38747509 -0.64632899  0.        ]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  


In [ ]:
#TODO: check if the new point is on the face